In [1]:
# POC for ny taxi

In [190]:
import pandas as pd
import glob
from pandasql import sqldf

In [108]:
# helpers
pysqldf = lambda q: sqldf(q, globals())

In [187]:
data_file_location = '../resources/data/enriched_trip_data.parquet'

## Loading enriched preprocessed data

In [188]:
# trip data
all_trip_data_df = None
for source_file_name in glob.glob(data_file_location):
    print(source_file_name)
    if all_trip_data_df is None:
        all_trip_data_df = pd.read_parquet(source_file_name)
    else:
        pd.concat([all_trip_data_df, pd.read_parquet(source_file_name)], ignore_index=True)

all_trip_data_df = all_trip_data_df.dropna()

../resources/data/enriched_trip_data.parquet


In [189]:
all_trip_data_df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_borough,pickup_zone,drop_off_borough,drop_off_zone,month
0,2019-02-01 00:59:04,2019-02-01 01:07:27,1,Manhattan,Clinton East,Manhattan,Union Sq,2019-02
1,2019-02-01 00:33:09,2019-02-01 01:03:58,1,Manhattan,Times Sq/Theatre District,Queens,Flushing Meadows-Corona Park,2019-02
2,2019-02-01 00:09:03,2019-02-01 00:09:16,1,Queens,Long Island City/Hunters Point,Queens,Long Island City/Hunters Point,2019-02
3,2019-02-01 00:45:38,2019-02-01 00:51:10,1,Queens,Forest Hills,Queens,Forest Hills,2019-02
4,2019-02-01 00:25:30,2019-02-01 00:28:14,1,Manhattan,Lenox Hill East,Manhattan,Yorkville West,2019-02
...,...,...,...,...,...,...,...,...
7019367,2019-02-28 23:18:52,2019-02-28 23:29:29,2,Manhattan,Clinton East,Manhattan,Midtown East,2019-02
7019368,2019-02-28 23:34:45,2019-02-28 23:42:11,1,Manhattan,Lenox Hill West,Manhattan,Upper East Side South,2019-02
7019369,2019-02-28 23:45:06,2019-02-28 23:57:21,1,Manhattan,Upper West Side South,Manhattan,Manhattanville,2019-02
7019370,2019-02-28 23:29:08,2019-02-28 23:29:11,1,Queens,Queensbridge/Ravenswood,Queens,Queensbridge/Ravenswood,2019-02


## Prepare Data according to Task 2.i

In [175]:
grouped = pysqldf("SELECT sum(passenger_count) as count, pickup_zone, drop_off_zone, month FROM all_trip_data_df GROUP BY month, pickup_zone, drop_off_zone")
grouped


,count,pickup_zone,drop_off_zone,month
0,1.0,Queensbridge/Ravenswood,Queensbridge/Ravenswood,2003-01
1,1.0,Central Park,Penn Station/Madison Sq West,2008-12
2,1.0,Clinton West,Midtown South,2008-12
3,1.0,Lenox Hill West,Upper East Side North,2008-12
4,1.0,Lincoln Square East,Sutton Place/Turtle Bay North,2008-12
...,...,...,...,...
29482,1.0,Battery Park City,Little Italy/NoLiTa,2020-05
29483,1.0,East Village,Stuy Town/Peter Cooper Village,2020-05
29484,1.0,Gramercy,Times Sq/Theatre District,2020-05
29485,1.0,SoHo,Lower East Side,2020-05


In [178]:
grouped[(grouped["pickup_zone"] == "Chinatown") & (grouped["month"] == "2019-10")].sort_values(by=['count'])

,count,pickup_zone,drop_off_zone,month
28458,1.0,Chinatown,Financial District North,2019-10
28459,2.0,Chinatown,Greenwich Village South,2019-10
28460,2.0,Chinatown,Union Sq,2019-10


In [182]:
# rank by k=50 per pickup_zone and month
ranked = grouped.copy(deep=True)
ranked["rank"]=ranked.groupby(["pickup_zone","month"])["count"].rank(method="max", ascending=False)
ranked = ranked[ranked["rank"] <= 50.0]
ranked[(ranked["pickup_zone"] == "Chinatown") & (ranked["month"] == "2019-10")].sort_values(by=['rank'])

,count,pickup_zone,drop_off_zone,month,rank
28459,2.0,Chinatown,Greenwich Village South,2019-10,2.0
28460,2.0,Chinatown,Union Sq,2019-10,2.0
28458,1.0,Chinatown,Financial District North,2019-10,3.0


In [169]:
# 2.i) Sample Queries
ranked[(ranked["pickup_zone"] == "Allerton/Pelham Gardens") & (ranked["rank"]==1.0)]

,count,pickup_zone,drop_off_zone,month,rank
124,12.0,Allerton/Pelham Gardens,Sunnyside,2019-09,1.0


In [186]:
# 2.i) Sample Queries
# Which destination was reached by the most passengers from Chinatown in September 2019.
# grouped[(grouped["rank"]==1.0) & (grouped["pickup_zone"] == "Chinatown")]
ranked[(ranked["pickup_zone"] == "Chinatown") & (ranked["rank"]<=5.0) & (ranked["month"]=="2019-09")]

,count,pickup_zone,drop_off_zone,month,rank
3998,930.0,Chinatown,East Village,2019-09,2.0
4003,974.0,Chinatown,Financial District North,2019-09,1.0
4047,677.0,Chinatown,Little Italy/NoLiTa,2019-09,5.0
4051,899.0,Chinatown,Lower East Side,2019-09,3.0
4118,815.0,Chinatown,TriBeCa/Civic Center,2019-09,4.0
